## This notebook provides the code for the paper - "Enhancing data security through ring fencing"

# Setting up the environment

In [1]:
!pip install sentence-transformers
!pip install cryptography
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=512a33567da20489890b212f7214189f4428e5185323240443824ef2dad3f1af
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.o

# Imports

In [34]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import spacy

from sklearn import preprocessing

from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertForTokenClassification, pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
import time

import unicodedata

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,T5Tokenizer, T5ForConditionalGeneration

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

import sqlite3
import predictionmodel

# EVALUATE

### Data preparation

In [4]:
input_query = "What is the bmi of the patient with id 1665 for Treatment"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
df_stroke = pd.read_csv("/content/drive/MyDrive/Ring_fencing_files/healthcare-dataset-stroke-data.csv")
df_stroke['bmi'].fillna(value = df_stroke['bmi'].mean(), inplace = True)

Setting up the sql database to run queries

In [36]:
connection = sqlite3.connect("stroke.db")
crsr = connection.cursor()
crsr.execute("""CREATE TABLE stroke_table(
   id                INTEGER NOT NULL PRIMARY KEY 
  ,gender            VARCHAR(6) NOT NULL
  ,age               NUMERIC(4,2) NOT NULL
  ,hypertension      BIT  NOT NULL
  ,heart_disease     BIT  NOT NULL
  ,ever_married      VARCHAR(3) NOT NULL
  ,work_type         VARCHAR(13) NOT NULL
  ,Residence_type    VARCHAR(5) NOT NULL
  ,avg_glucose_level NUMERIC(6,2) NOT NULL
  ,bmi               NUMERIC(4,1) NOT NULL
  ,smoking_status    VARCHAR(15) NOT NULL
  ,stroke            BIT  NOT NULL
);""")

file = open("/content/drive/MyDrive/Ring_fencing_files/stroke.txt", "r")
queries = file.readlines()
for i in queries:
  crsr.execute(i[:-1])

In [37]:
# Defining the databases. Here we have stored the data in the form of python classes and dictionaries 

# This information will be provided by the hospital's data. This mock data has only been created for the purpose of this demonstration

class User:
  def __init__(self, id, name, institution, role, location):
    self.id = id
    self.name = name
    self.institution = institution
    self.role = role
    self.location = location

purpose_list_insurance = ["Policy design", "Risk assessment", "Customer service", "Fraud detection", "Claims processing"]
purpose_list_hospital = ["Treatment", "Reference", "Emergency", "Patient inquiry"]
purpose_list_research = ["New model", "Paper publish"]
institution_list = ["INSURANCE", "HOSPITAL", "RESEARCH"]

# list of roles in each institution
roles_institutions = {"INSURANCE": ["Actuarial_Analyst", "Insurance_Salesman", "Admin", "Investigator"],
                      "HOSPITAL": ["Primary_Doctor", "Secondary_Doctor", "Admin", "Nurse"],
                      "RESEARCH": ["Research_Intern", "Post_Doc_Fellow", "Professor"]}

# Query types allowed for institution
institution_query_types = {"INSURANCE": ["ANALYTICS", "INDIVIDUAL"], "HOSPITAL": ["ANALYTICS", "INDIVIDUAL", "PREDICTION"], "RESEARCH": ["ANALYTICS", "INDIVIDUAL"]}

# Query types allowed for roles
purpose_role_insurance = {"Actuarial_Analyst": ["ANALYTICS"], "Insurance_Salesman": [], "Admin": ["ANALYTICS"], "Investigator": ["ANALYTICS", "INDIVIDUAL"]}
purpose_role_hospital = {"Primary_Doctor": ["ANALYTICS", "INDIVIDUAL"], "Secondary_Doctor": ["ANALYTICS", "INDIVIDUAL"], "Admin": ["ANALYTICS", "INDIVIDUAL"], "Nurse": []}
purpose_role_research = {"Research_Intern": ["INDIVIDUAL", "ANALYTICS"], "Post_Doc_Fellow": ["ANALYTICS"], "Professor": ["ANALYTICS"]}

# Purpose table: "Purpose": [{"roles": "types of competitor"}, "time_range", "locations"]
# purpose table for insurance company
purpose_table_insurance = {"Risk Assessment": [[], [0, 0], []],
                          "Policy design": [{"Actuarial_Analyst": ["ANALYTICS"]}, [9, 18], ["Office"]],
                          "Customer service": [{"Admin": ["ANALYTICS"]}, [9, 18], ["Office"]],
                          "Fraud detection": [{"Investigator": ["ANALYTICS", "INDIVIDUAL"]}, [9, 18], ["Office"]],
                          "Claims processing": [{"Investigator": ["ANALYTICS", "INDIVIDUAL"]}, [9, 18], ["Office"]]}

# purpose table for hospital
purpose_table_hospital = {"Treatment": [{"Primary_Doctor": ["INDIVIDUAL", "PREDICTION"]}, [0, 24], ["Hospital", "Home"]],
                          "Reference": [{"Secondary_Doctor": ["INDIVIDUAL"]}, [0, 24], ["Hospital", "Home"]],
                          "Emergency": [{"Primary_Doctor": ["INDIVIDUAL", "PREDICTION"], "Secondary_Doctor": ["INDIVIDUAL", "PREDICTION"]}, [0, 24], ["Hospital"]],
                          "Patient inquiry": [{"Admin": ["INDIVIDUAL", "ANALYTICS"]}, [9, 18], ["Hospital"]]}

# purposes table for research
purpose_table_research = {"Treatment": [{"Research_Intern": ["INDIVIDUAL", "ANALYTICS"]}, [9, 18], ["Reasearch Lab"]],
                          "Paper publish": [{"Professor": ["ANALYTICS"], "Post_Doc_Fellow": ["ANALYTICS"]}, [9, 18], ["Research Lab"]]}

curr_user = User(1, "Annie", "HOSPITAL", "Primary_Doctor", "Hospital")

In [38]:
# Load the data
df_classify = pd.read_csv("/content/drive/MyDrive/Ring_fencing_files/Final_query_dataset - Sheet1.csv")

In [39]:
df_classify_analysis = pd.read_csv("/content/drive/MyDrive/Ring_fencing_files/Analysis_Classification_dataset - Sheet1.csv")

### Extracting the purpose through similarity matching

In [40]:
# Setting up the purpose list
purpose_list = None
if curr_user.institution == "INSURANCE":
  purpose_list = purpose_list_insurance

if curr_user.institution == "HOSPITAL":
  purpose_list = purpose_list_hospital

if curr_user.institution == "RESEARCH":
  purpose_list = purpose_list_research

In [64]:
model = SentenceTransformer('bert-base-nli-mean-tokens')
nlp = spacy.load("en_core_web_sm")

query = unicodedata.normalize('NFKD', input_query)
print("=================================")
print("Before breaking")
print("=================================")
print(query)

lower_query = query.lower()
q_purpose = ""

break_ind = -1
tags = []
tokens = []
purpose_found = False

# POS tagging
doc = nlp(query)
for token in doc:
    tokens.append(str(token))
    tags.append((str(token), token.pos_))

f = -1
purpose_ = ""
start_time = time.time()
# Extracting purpose and breaking
for purpose in purpose_list:
  p = purpose.lower()
  doc_purp = nlp(purpose)
  token_purp = []
  for i in doc_purp:
    token_purp.append(str(i))
  ngram = len(token_purp)
  for i in range(len(tokens) - 1, ngram - 1, -1):
    str1 = " ".join(tokens[i - ngram + 1: i + 1])
    str2 = " ".join(token_purp)
    samples = [str1, str2]
    sentence_embeddings = model.encode(samples)
    sim = cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:])
    if float(sim[0][0]) > 0.9:
      f = i - ngram + 1
  if(f == -1):
    continue
  purpose_ = purpose 
  break
end_time = time.time()
print("Time taken to extract the purpose through similarity matching: " + str(end_time - start_time))


purpose = purpose_
start_time = time.time()
# If purpose not found
if f == -1:
  purpose = "General Purpose"
  seg1 = query  

else:
  # Purpose found
  purpose_found = True
  print("Purpose = " + purpose)
  print()

  # Breaking the query
  doc = nlp(query)
  seg1 = ""

  # Rule based query division
  for i in range(len(doc) - 1, -1, -1):
    # print(i)
    if tags[i][1] == "ADP" or tags[i][0].lower() == "to":
      seg1 = tokens[:i]

      # Printing the Second Segement
      seg2 = tokens[i:]
      break

end_time = time.time()
query_without_purpose = " ".join(seg1)
extracted_purpose = purpose
print("Time taken to extract purpose: " + str(end_time - start_time))

Before breaking
What is the bmi of the patient with id 1665 for Treatment
Time taken to extract the purpose through similarity matching: 0.9815773963928223
Purpose = Treatment

Time taken to extract purpose: 0.010278463363647461


### Classifying the query

In [42]:
def get_bert_emb(data_frame):
  """
  Input a data frame and return the bert embedding vectors for the each sentence column.
  Return 2 matrices each of shape (#_samples, #size_of_word_emb).
  """
  bert_model = SentenceTransformer('distilbert-base-uncased')
  
  feature = bert_model.encode(data_frame)
  
  return feature

In [43]:
#Training the classification model

for i in df_classify.index:
  if df_classify["LABEL"][i] == "ANALYTICS":
    df_classify["LABEL"][i] = 0
  if df_classify["LABEL"][i] == "INDIVIDUAL":
    df_classify["LABEL"][i] = 1
  if df_classify["LABEL"][i] == "PREDICTION":
    df_classify["LABEL"][i] = 2

y_train_classify = df_classify["LABEL"]
X_train_classify = df_classify["QUERY"]

classification_embeddings = get_bert_emb(np.array(X_train_classify))
query_classifier = LogisticRegression(max_iter = 500)
query_classifier.fit(np.array(classification_embeddings), np.array(y_train_classify, dtype = "int"))

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


LogisticRegression(max_iter=500)

In [44]:
start_time = time.time()
query_class = query_classifier.predict(get_bert_emb(query_without_purpose).reshape(1, -1))
end_time = time.time()
print("Time taken to classify query: " + str(end_time - start_time))
query_class_text = ""
if query_class == 0:
  query_class_text = "ANALYTICS"
elif query_class == 1:
  query_class_text = "INDIVIDUAL"
else:
  query_class_text = "PREDICTION"

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Time taken to classify query: 2.1404991149902344


In [45]:
# predicting Analysis class
query_analysis_class = None
if query_class == 0:
  for i in df_classify_analysis.index:
    if df_classify_analysis["LABEL"][i] == "COUNT":
      df_classify_analysis["LABEL"][i] = 0
    # The probability class also includes queries related to percentages
    if df_classify_analysis["LABEL"][i] == "PROBABILITY":
      df_classify_analysis["LABEL"][i] = 1
    if df_classify_analysis["LABEL"][i] == "CORRELATION":
      df_classify_analysis["LABEL"][i] = 2
  y_train_classify = df_classify_analysis["LABEL"]
  X_train_classify = df_classify_analysis["QUERY"]

  analysis_class_emb = get_bert_emb(np.array(X_train_classify))
  ana_classifier = LogisticRegression(max_iter = 500)
  ana_classifier.fit(np.array(analysis_class_emb), np.array(y_train_classify, dtype = "int"))
  start_time = time.time()
  query_analysis_class = ana_classifier.predict(get_bert_emb(query_without_purpose).reshape(1, -1))
  end_time = time.time()
  print("Time taken to run analytics classification" + str(end_time - start_time))

In [46]:
query_analysis_class_text = ""
if query_analysis_class == 0:
  query_analysis_class_text = "COUNT"
elif query_analysis_class == 1:
  query_analysis_class_text = "PROBABILITY"
elif query_analysis_class == 2:
  query_analysis_class_text = "CORRELATION"
print("Query class: " + query_class_text)
print("Analysis class: " + query_analysis_class_text)

Query class: INDIVIDUAL
Analysis class: 


# EXECUTE - Encryption

## Encrypting the general data

In [47]:
start_time = time.time()

total_msg = "No message to be displayed"

if query_class == 0:
  total_count_stroke = df_stroke["stroke"].value_counts()[1]
  total_percentage_stroke = df_stroke["stroke"].value_counts()[1] / (df_stroke["stroke"].value_counts()[1] + df_stroke["stroke"].value_counts()[0])
  count_msg = "Number of people in the total population with stroke " + str(total_count_stroke)
  percentage_msg = "Percentage of people in the total population with stroke " + str(total_percentage_stroke)
  total_msg = count_msg + "\n" + percentage_msg

if query_class == 1:
  given_id = 1665
  ind = -1
  for i in df_stroke.index:
    if df_stroke["id"][i] == given_id:
      ind = i
      break
  if ind == -1:
    print("No patient found with the given id")
  else:
    total_msg = "Gender: " + str(df_stroke["gender"][ind]) + ", Age: " + str(df_stroke["age"][ind])


total_msg = total_msg.encode()

key_role = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)


ciphertext_role = key_role.public_key().encrypt(
    total_msg,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

end_time = time.time()
print("Time taken to encrypt general role fence data " + str(end_time - start_time))

Time taken to encrypt general role fence data 0.044239044189453125


## NL2SQL

In [66]:
#Loading the model

def run_nl2sql(query):
  tokenizer = AutoTokenizer.from_pretrained("tscholak/3vnuv1vf") 
  model = AutoModelForSeq2SeqLM.from_pretrained("tscholak/3vnuv1vf")
  pipe = pipeline('translation_xx_to_yy', model=model, tokenizer=tokenizer)
  input = query + " | strokeDB | stroke_table : id, gender, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status, stroke"
  start_time = time.time()
  output = pipe(input)[0]['translation_text']
  end_time = time.time()
  return output, end_time - start_time

## Analysis functions

In [49]:
def get_gender_analysis(df):
  count_male = 0
  count_female = 0
  count_other = 0
  men = df["gender"].value_counts()["Male"]
  women = df["gender"].value_counts()["Female"]
  other = df["gender"].value_counts()["Male"]
  for i in df.index:
    if df["gender"][i] == "Male" and df["stroke"][i] == 1:
      count_male += 1
    if df["gender"][i] == "Female" and df["stroke"][i] == 1:
      count_female += 1
    if df["gender"][i] == "Other" and df["stroke"][i] == 1:
      count_other += 1
  
  men_c = "Number of men with stroke " + str(count_male)
  women_c = "Number of wormen with stroke " + str(count_female)
  other_c = "Number of people belonging to other genders with stroke " + str(count_other)

  men_p = "Percentage of men with stroke " + str(count_male*100 / men)
  women_p = "Percentage of wormen with stroke " + str(count_female*100 / women)
  other_p = "Percentage of people belonging to other genders with stroke " + str(count_other*100 / other)
  return (men_c + "\n" + women_c + "\n" + other_c, men_p + "\n" + women_p + "\n" + other_p)

def get_age_analysis(df):
  count_age = [0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0]
  age = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  
  for i in df.index:
    val = df["age"][i]
    if val < 10:
      age[0] += 1
      if df["stroke"][i] == 1:
        count_age[0] += 1
    elif val >= 10 and val < 20:
      age[1] += 1
      if df["stroke"][i] == 1:
        count_age[1] += 1
    elif val >= 20 and val < 30:
      age[2] += 1
      if df["stroke"][i] == 1:
        count_age[2] += 1
    elif val >= 30 and val < 40:
      age[3] += 1
      if df["stroke"][i] == 1:
        count_age[3] += 1
    elif val >= 40 and val < 50:
      age[4] += 1
      if df["stroke"][i] == 1:
        count_age[4] += 1
    elif val >= 50 and val < 60:
      age[5] += 1
      if df["stroke"][i] == 1:
        count_age[5] += 1
    elif val >= 60 and val < 70:
      age[6] += 1
      if df["stroke"][i] == 1:
        count_age[6] += 1
    elif val >= 70 and val < 80:
      age[7] += 1
      if df["stroke"][i] == 1:
        count_age[7] += 1
    elif val >= 80 and val < 90:
      age[8] += 1
      if df["stroke"][i] == 1:
        count_age[8] += 1
    elif val >= 90:
      age[9] += 1
      if df["stroke"][i] == 1:
        count_age[9] += 1

  count_ana = "Age Counts (stroke)"
  perc_ana = "Age percentage (stroke)"
  t = 0
  for i in range(10):
    count_ana += "Range " + str(t) + "-" + str(t + 10) + ": " + str(count_age[i]) + "%"
    perc_ana += "Range " + str(t) + "-" + str(t + 10) + ": " + str(count_age[i]*100 / age[i]) + "%"
  return (count_ana, perc_ana)

def get_hypertension_analysis(df):
  count_hyp = 0
  count_not_hyp = 0

  hyp = df["hypertension"].value_counts()[1]
  not_hyp = df["hypertension"].value_counts()[0]
  for i in df.index:
    if df["hypertension"][i] == 1 and df["stroke"][i] == 1:
      count_hyp += 1
    if df["hypertension"][i] == 0 and df["stroke"][i] == 1:
      count_not_hyp += 1
  hyp_c = "Number of people with hypertension with stroke " + str(count_hyp)
  not_hyp_c = "Number of people without hypertension with stroke " + str(count_not_hyp)

  hyp_p = "Percentage of people with hypertension with stroke " + str(count_hyp*100 / hyp)
  not_hyp_p = "Percentage of people without hypertension without stroke" + str(count_not_hyp*100 / not_hyp)
  return tuple((hyp_c + "\n" + not_hyp_c, hyp_p + "\n" + not_hyp_p))

def get_heart_disease_analysis(df):
  count_heart = 0
  count_not_heart = 0

  hyp = df["heart_disease"].value_counts()[1]
  not_hyp = df["heart_disease"].value_counts()[0]
  for i in df.index:
    if df["heart_disease"][i] == 1 and df["stroke"][i] == 1:
      count_heart += 1
    if df["heart_disease"][i] == 0 and df["stroke"][i] == 1:
      count_not_heart += 1
  heart_c = "Number of people with heart_disease with stroke " + str(count_heart)
  not_heart_c = "Number of people without heart_disease with stroke " + str(count_not_heart)

  heart_p = "Percentage of people with heart_disease with stroke " + str(count_heart*100 / hyp)
  not_heart_p = "Percentage of people without heart_disease without stroke" + str(count_not_heart*100 / not_hyp)
  return (heart_c + "\n" + not_heart_c, heart_p + "\n" + not_heart_p)

def get_married_analysis(df):
  count_marr = 0
  count_not_marr = 0

  heart = df["ever_married"].value_counts()["Yes"]
  not_heart = df["ever_married"].value_counts()["No"]
  for i in df.index:
    if df["ever_married"][i] == "Yes" and df["stroke"][i] == 1:
      count_marr += 1
    if df["ever_married"][i] == "No" and df["stroke"][i] == 1:
      count_not_marr += 1
  marr_c = "Number of people who have ever married with stroke " + str(count_marr)
  not_marr_c = "Number of people who have never married with stroke " + str(count_not_marr)

  marr_p = "Percentage of people who have ever married with stroke " + str(count_marr*100 / heart)
  not_marr_p = "Percentage of people who have never married without stroke" + str(count_not_marr*100 / not_heart)
  return (marr_c + "\n" + not_marr_c, marr_p + "\n" + not_marr_p)

def get_living_analysis(df):
  count_urban = 0
  count_not_urban = 0

  urb = df["Residence_type"].value_counts()["Urban"]
  not_urb = df["Residence_type"].value_counts()["Rural"]
  for i in df.index:
    if df["Residence_type"][i] == "Urban" and df["stroke"][i]== 1:
      count_urban += 1
    if df["Residence_type"][i] == "Rural" and df["stroke"][i]== 1:
      count_not_urban += 1
  urban_c = "Number of people with Residence_type urban with stroke " + str(count_urban)
  not_urban_c = "Number of people with Residence_type rural with stroke " + str(count_not_urban)

  urban_p = "Percentage of people with Residence_type urbam with stroke " + str(count_urban*100 / urb)
  not_urban_p = "Percentage of people with Residence_type rural without stroke" + str(count_not_urban*100 / not_urb)
  return (urban_c + "\n" + not_urban_c, urban_p + "\n" + not_urban_p)

def get_smoke_analysis(df):
  count_smoke = 0
  count_not_smoke = 0

  smoke = df["smoking_status"].value_counts()["formerly smoked"]
  not_smoke = df["smoking_status"].value_counts()["never smoked"]
  for i in df.index:
    if df["smoking_status"][i] == "formerly smoked" and df["stroke"][i] == 1:
      count_smoke += 1
    if df["smoking_status"][i] == "never smoked" and df["stroke"][i] == 1:
      count_not_smoke += 1
  smoke_c = "Number of people who have smoked with stroke " + str(count_smoke)
  not_smoke_c = "Number of people who have never smoked with stroke " + str(count_not_smoke)

  smoke_p = "Percentage of people who have smoked with stroke " + str(count_smoke*100 / smoke)
  not_smoke_p = "Percentage of people who have never smoked without stroke" + str(count_not_smoke*100 / not_smoke)
  return (smoke_c + "\n" + not_smoke_c, smoke_p + "\n" + not_smoke_p)

def get_bmi_analysis(df):
  count_bmi = [0, 0, 0]
  bmi = [0, 0, 0]
  
  for i in df.index:
    val = df["bmi"][i]
    if val < 20:
      bmi[0] += 1
      if df["stroke"][i] == 1:
        count_bmi[0] += 1
    elif val >= 20 and val < 25:
      bmi[1] += 1
      if df["stroke"][i] == 1:
        count_bmi[1] += 1
    elif val >= 25:
      bmi[2] += 1
      if df["stroke"][i] == 1:
        count_bmi[2] += 1

  count_ana = "bmi Counts (stroke)\n"
  perc_ana = "bmi percentage (stroke)\n"
  count_ana += "Range 0 - 20 : " + str(count_bmi[0]) + "%\n"
  perc_ana += "Range 0 - 20 : " + str(count_bmi[0]*100 / bmi[0]) + "%\n"
  count_ana += "Range 20-25 : " + str(count_bmi[1]) + "%\n"
  perc_ana += "Range 20-25 : " + str(count_bmi[1]*100 / bmi[1]) + "%\n"
  count_ana += "Range > 25 : " + str(count_bmi[2]) + "%"
  perc_ana += "Range > 25 : " + str(count_bmi[2]*100 / bmi[2]) + "%"
    
  return (count_ana, perc_ana)

def get_work_analysis(df):
  count_work = [0, 0, 0]
  work = [0, 0, 0]
  
  for i in df.index:
    val = df["work_type"][i]
    if val == "Private":
      work[0] += 1
      if df["stroke"][i] == 1:
        count_work[0] += 1
    elif val == "Self-employed":
      work[1] += 1
      if df["stroke"][i] == 1:
        count_work[1] += 1
    elif val == "Other":
      work[2] += 1
      if df["stroke"][i] == 1:
        count_work[2] += 1

  count_ana = "work Counts (stroke)\n"
  perc_ana = "work percentage (stroke)\n"
  count_ana += "Private: " + str(count_work[0]) + "%\n"
  perc_ana += "Private: " + str(count_work[0]*100 / work[0]) + "%\n"
  count_ana += "Self-employed : " + str(count_work[1]) + "%\n"
  perc_ana += "Self-employed : " + str(count_work[1]*100 / work[1]) + "%\n"
  count_ana += "Other : " + str(count_work[2]) + "%"
  perc_ana += "Other : " + str(count_work[2]*100 / work[2]) + "%"
    
  return (count_ana, perc_ana)

def get_glucose_analysis(df):
  count_glucose = [0, 0, 0]
  Glucose = [0, 0, 0]
  
  for i in df.index:
    val = df["avg_glucose_level"][i]
    if val < 50:
      Glucose[0] += 1
      if df["stroke"][i] == 1:
        count_glucose[0] += 1
    elif val >= 50 and val < 100:
      Glucose[1] += 1
      if df["stroke"][i] == 1:
        count_glucose[1] += 1
    elif val >= 100 and val < 150:
      Glucose[2] += 1
      if df["stroke"][i] == 1:
        count_glucose[2] += 1
    elif val >= 150:
      Glucose[3] += 1
      if df["stroke"][i] == 1:
        count_glucose[3] += 1

  count_ana = "Glucose Counts (stroke)\n"
  perc_ana = "Glucose percentage (stroke)\n"
  count_ana += "Range 0 - 50 : " + str(count_glucose[0]) + "%\n"
  perc_ana += "Range 0 - 50 : " + str(count_glucose[0]*100 / Glucose[0]) + "%\n"
  count_ana += "Range 50-100 : " + str(count_glucose[1]) + "%\n"
  perc_ana += "Range 50-100 : " + str(count_glucose[1]*100 / Glucose[1]) + "%\n"
  count_ana += "Range 100-150 : " + str(count_glucose[2]) + "%\n"
  perc_ana += "Range 100-150 : " + str(count_glucose[2]*100 / Glucose[2]) + "%\n"
  count_ana += "Range >= 150 : " + str(count_glucose[3]) + "%"
  perc_ana += "Range >= 150 : " + str(count_glucose[3]*100 / Glucose[3]) + "%"
    
  return (count_ana, perc_ana)



In [50]:
def get_analysis(column, df):
  if column == "gender":
    out = get_gender_analysis(df)
    return out
  elif column == "age":
    out = get_age_analysis(df)
    return out
  elif column == "hypertension":
    out = get_hypertension_analysis(df)
    return tuple(out)
  elif column == "heart_disease":
    out = get_heart_disease_analysis(df)
    return out
  elif column == "ever_married":
    out = get_married_analysis(df)
    return out
  elif column == "bmi":
    out = get_bmi_analysis(df)
    return out
  elif column == "smoking_status":
    out = get_smoke_analysis(df)
    return out
  elif column == "Residence_type":
    out = get_living_analysis(df)
    return out
  elif column == "work_type":
    out = get_work_analysis(df)
    return out
  elif column == "avg_glucose_level":
    out = get_glucose_analysis(df)
    return out

def get_correlation(column1, column2, df):
  return df.corr()[column1][column2]

## Encrypting final outputs

In [67]:
final_output = ""


# Analytics
if query_class == 0:
  data = ""
  if query_analysis_class == 1:
    for i in df_stroke.columns:
      column_name = i.replace("_", " ")
      if column_name.lower() in input_query.lower()and i != "stroke":
        temp = get_analysis(i, df_stroke)
        data += (temp[1] + "\n")
  elif query_analysis_class == 0:
    for i in df_stroke.columns:
      column_name = i.replace("_", " ")
      if column_name.lower() in input_query.lower() and i != "stroke":
        temp = get_analysis(i, df_stroke)
        data += (temp[0] + "\n")
  else:
    column1 = ""
    column2 = ""
    for i in df_stroke.columns:
      column_name = i.replace("_", " ")
      if column_name in input_query.lower():
        if column1 == "":
          column1 = i
        elif column1 != "" and column2 == "":
          column2 = i
      if column1 != "" and column2 != "":
        break
    data = "Correlation between {0} and {1}: {2}".format(column1, column2, get_correlation(column1, column2, df_stroke))
  final_output = data

if query_class == 1:
  sql_query, t = run_nl2sql(query_without_purpose)
  print("Time taken to run nl2sql: " + str(t))
  start_time = time.time()
  crsr.execute(sql_query[11:])
  end_time = time.time()
  print("Time taken to fetch the query: " + str(end_time - start_time))
  final_output = str(crsr.fetchall())


if query_class == 2:
  df_stroke_pred = df_stroke.copy()
  df_stroke_pred.drop('id', axis = 1, inplace =True)
  input_array = []
  # Input order
  # gender, age, hypertension, heart_disease,	ever_married,	work_type,	Residence_type,	avg_glucose_level,	bmi,	smoking_status
  input_array.append(input("Enter the gender: "))
  input_array.append(int(input("Enter the age: ")))
  input_array.append(int(input("Does the patient have hypertension? (0/1): ")))
  input_array.append(int(input("Does the patient have heart_disease? (0/1): ")))
  input_array.append(input("Has the patient ever married? (Yes/No): "))
  input_array.append(input("Patient work type: "))
  input_array.append(input("Patient residence type (Urban / Rural): "))
  input_array.append(float(input("average glucose level: ")))
  input_array.append(float(input("BMI: ")))
  input_array.append(input("Smoking status: "))


  # Encode Categorical Data
  label_encoder = preprocessing.LabelEncoder()

  label_encoder.fit(df_stroke_pred['gender'])
  input_array[0] = label_encoder.transform([input_array[0]])[0]
  df_stroke_pred['gender'] = label_encoder.transform(df_stroke_pred["gender"])

  label_encoder.fit(df_stroke_pred['ever_married'])
  input_array[4] = label_encoder.transform([input_array[4]])[0]
  df_stroke_pred['ever_married'] = label_encoder.transform(df_stroke_pred["ever_married"])

  label_encoder.fit(df_stroke_pred['work_type'])
  input_array[5] = label_encoder.transform([input_array[5]])[0]
  df_stroke_pred['work_type'] = label_encoder.transform(df_stroke_pred["work_type"])

  label_encoder.fit(df_stroke_pred['Residence_type'])
  input_array[6] = label_encoder.transform([input_array[6]])[0]
  df_stroke_pred['Residence_type'] = label_encoder.transform(df_stroke_pred["Residence_type"])

  label_encoder.fit(df_stroke_pred['smoking_status'])
  input_array[9] = label_encoder.transform([input_array[9]])[0]
  df_stroke_pred['smoking_status'] = label_encoder.transform(df_stroke_pred["smoking_status"])

  df_stroke['bmi'].fillna(value = df_stroke['bmi'].mean(), inplace = True)

  model = predictionmodel.train_model(df_stroke_pred)
  prediction = predictionmodel.predict(model, [input_array])[0]
  if prediction == 1:
    final_output = "The user will have a stroke"
  else:
    final_output = "The user will not have a stroke"

final_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

ciphertext_final = final_key.public_key().encrypt(
    final_output.encode(),
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)



Time taken to run nl2sql: 23.77999472618103
Time taken to fetch the query: 5.2928924560546875e-05


# Enforce

In [52]:
# Setting up the purpose table
purpose_table = None
purpose_role_table = None
if curr_user.institution == "INSURANCE":
  purpose_table = purpose_table_insurance
  purpose_role_table = purpose_role_insurance

if curr_user.institution == "HOSPITAL":
  purpose_table = purpose_table_hospital
  purpose_role_table = purpose_role_hospital

if curr_user.institution == "RESEARCH":
  purpose_table = purpose_table_research
  purpose_role_table = purpose_role_research

In [53]:
# Collecting keys to decrypt data
keys_collected = []
decryption_keys = []

Checking the role and institution

In [54]:
start_time = time.time()
inst_fence = False
role_fence = False

if query_class_text in institution_query_types[curr_user.institution]:
  inst_fence = True

if query_class_text in purpose_role_table[curr_user.role]:
  role_fence = True

end_time = time.time()
print("Time taken for role and institution fence: ", str(end_time - start_time))

if not inst_fence:
  print("Access denied at institute fence")

if not inst_fence and role_fence:
  print("Access denied at role fence")

Time taken for role and institution fence:  0.00023508071899414062


In [55]:
if role_fence and inst_fence:
  keys_collected.append("role_key")

# key = Fernet.generate_key()
# fernet = Fernet(key)
# enc_total_count = fernet.encrpyt(total_count_stroke.encode())
# decryption_keys["total_count"] = [enc_total_count, key]

# key = Fernet.generate_key()
# fernet = Fernet(key)
# enc_total_percentage = fernet.encrypt(total_percentage_stroke.encode())
# decryption_keys["total_percentage"] = [enc_total_percentage, key]
# decryption_keys["role_layer": [ciphertext_role, key_role]]

Checking the purpose

In [56]:
contracts = purpose_table[extracted_purpose]

In [57]:
start_time = time.time()
purpose_fence = False
if curr_user.role in contracts[0] and query_class_text in contracts[0][curr_user.role]:
  purpose_fence = True

end_time = time.time()
print("Time taken at purpose fence: " + str(end_time - start_time))

if not purpose_fence:
  print("Access denied at purpose fence")

Time taken at purpose fence: 0.00022220611572265625


Location and time conditions

In [58]:
time_fence = False
location_fence = False

start_time = time.time()
if curr_user.location in contracts[2]:
  location_fence = True

curr_time = int(time.strftime("%H"))

if curr_time > contracts[1][0] and curr_time < contracts[1][1]:
  time_fence = True
end_time = time.time()

print("Time taken at time and location fences: " + str(end_time - start_time))

if not time_fence:
  print("Failed at time fence")
if not location_fence:
  print("Failed at location fence")

Time taken at time and location fences: 0.0002899169921875


In [59]:
if purpose_fence and time_fence and location_fence and role_fence and inst_fence:
  keys_collected.append("final_key")

# EXECUTE - Decryption

## Decrypting the data

In [60]:
def decrypt_data(ciphertext, key):

  plaintext = key.decrypt(
      ciphertext,
      padding.OAEP(
          mgf=padding.MGF1(algorithm=hashes.SHA256()),
          algorithm=hashes.SHA256(),
          label=None
      )
  )
  return plaintext.decode()

In [61]:
def release_general_analysis(decryption_keys):
  key = decryption_keys["role_layer"][1]
  ciphertext = decryption_keys["role_layer"][0]
  plaintext = key.decrypt(
      ciphertext,
      padding.OAEP(
          mgf=padding.MGF1(algorithm=hashes.SHA256()),
          algorithm=hashes.SHA256(),
          label=None
      )
  )
  print(plaintext.decode())

## Releasing the data

In [62]:
# Releasing the data

start_time = time.time()
if query_class == 0 or query_class == 1:
  # ANALYSIS
  if "role_key" in keys_collected:
    print(decrypt_data(ciphertext_role, key_role))

if "final_key" in keys_collected:
  print(decrypt_data(ciphertext_final, final_key))
end_time = time.time()
print("Time taken to decrypt and release the data: " + str(end_time - start_time))


Gender: Female, Age: 79.0
[(24,)]
Time taken to decrypt and release the data: 0.014449119567871094
